<a href="https://colab.research.google.com/github/jai-sundaram/ml_tutorial/blob/main/my_data_preprocessing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing Tools

## Importing the libraries

In [1]:
##numpy helps us work with arrays
import numpy as np
#matplotlib helps us plot numerous charts, particularly pylpot which allows us to plot nice charts
import matplotlib.pyplot as plt
#pandas helps import the dataset, create the matrix, and independent variable vector
import pandas as pd


#to add the csv to the google colab notebook, click on the Files folder, click on the Upload button, and then upload the file

## Importing the dataset

In [3]:
#reading the values of the data set and creating a data frame
#make sure the path for the csv is correct
#find the csv in the folder structure, and click on copy path
dataset = pd.read_csv('/Data.csv')
#creating two additional entities
#creating a matrix of features and a dependent variable vector

#the features are the columns you are using to predict the dependent variable
#the dependent variable is the last column, the thing you are predicting
#in other terms, the features are the independent variables

##in this case, the country, age, and salary are the features/dependent variabels
##the fact if they purchased or not is the dependent variable

#formatting wise, most datasets will have the features/dependent variables in the first few columns, and the dependent variable in the last column
#to get the create the matrix, using the iloc method
#using the indexes of the columns
#getting all the rows, so just say ":", which is everything, since there is no specified range
#getting the first three columns, so basically getting all rows except the last one
#the last one is -1, so just do :-1
#including everything in the lower bound, excluding the upper bound which is -1, or the last value
#adding .values, means that we are just simply taking the values
x = dataset.iloc[:,:-1].values

#doing the same for the dependent variable vector
#only need to get one column, so not a range
#just -1
y= dataset.iloc[:, -1].values

In [4]:
#testing to make sure we correctly created the matrix and vector
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [5]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

## Encoding categorical data

### Encoding the Independent Variable

### Encoding the Dependent Variable

## Splitting the dataset into the Training set and Test set

## Feature Scaling